In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy
import statsmodels.stats.multitest as smm
from statsmodels.stats.weightstats import *
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## 1.

### В данном задании вам нужно будет

* проанализировать АБ тест, проведенный на реальных пользователях Яндекса
* подтвердить или опровергнуть наличие изменений в пользовательском поведении между контрольной (control) и тестовой (exp) группами
* определить характер этих изменений и практическую значимость вводимого изменения
* понять, какая из пользовательских групп более всего проигрывает / выигрывает от тестируемого изменения (локализовать изменение)

### Описание данных:

* userID: уникальный идентификатор пользователя
* browser: браузер, который использовал userID
* slot: в каком статусе пользователь участвовал в исследовании (exp = видел измененную страницу, control = видел неизменную страницу)
* n_clicks: количество кликов, которые пользоваль совершил за n_queries
* n_queries: количество запросов, который совершил userID, пользуясь браузером browser
* n_nonclk_queries: количество запросов пользователя, в которых им не было совершено ни одного клика

Обращаем ваше внимание, что не все люди используют только один браузер, поэтому в столбце userID есть повторяющиеся идентификаторы. В предлагаемых данных уникальным является сочетание userID и browser.

Основная метрика, на которой мы сосредоточимся в этой работе, — это количество пользовательских кликов на web-странице в зависимости от тестируемого изменения этой страницы.

Посчитайте, насколько в группе exp больше пользовательских кликов по сравнению с группой control в процентах от числа кликов в контрольной группе.

Полученный процент округлите до третьего знака после точки.

In [3]:
data = pd.read_csv("ab_browser_test.csv", index_col=False) 
data.head()

,userID,browser,slot,n_clicks,n_queries,n_nonclk_queries
0,1,Browser #2,exp,23,32,19
1,3,Browser #4,exp,3,4,2
2,5,Browser #4,exp,29,35,16
3,6,Browser #4,control,12,6,0
4,7,Browser #4,exp,54,68,30


In [15]:
cont_exp_agr = pd.pivot_table(data,index='slot',values='n_clicks',aggfunc=np.sum)

In [32]:
perc = (np.float128(cont_exp_v['exp'])\
        /np.float128(cont_exp_v['control'])-1)*100
print round(perc,3)

1.614


## 2.

Давайте попробуем посмотреть более внимательно на разницу между двумя группами
(control и exp) относительно количества пользовательских кликов.

Для этого постройте с помощью бутстрепа 95% доверительный интервал для средних
значений и медиан количества кликов в каждой из двух групп. Отметьте все верные
утверждения.

In [76]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def iter_bootstrap_samples(data, n_samples):
    for i in xrange(n_samples):
        indices = np.random.randint(0, len(data),len(data))
        yield data[indices]
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [42]:
exp_slot = data.n_clicks[data.slot == 'exp'].values
control_slot = data.n_clicks[data.slot == 'control'].values

In [82]:
%%time
np.random.seed(0)

exp_median_scores = map(np.median, iter_bootstrap_samples(exp_slot, 1500))
control_median_scores = map(np.median, iter_bootstrap_samples(control_slot, 1500))

print "95% confidence interval for the exp median repair time:",  stat_intervals(exp_median_scores, 0.05)
print "95% confidence interval for the control median repair time:",  stat_intervals(control_median_scores, 0.05)

95% confidence interval for the exp median repair time: [ 5.  5.]
95% confidence interval for the control median repair time: [ 4.  4.]
CPU times: user 24.1 s, sys: 24 ms, total: 24.1 s
Wall time: 24.1 s


In [84]:
delta_median_scores = map(lambda x: x[1] - x[0], zip(exp_median_scores, control_median_scores))
print "95% confidence interval for the difference between medians",  stat_intervals(delta_median_scores, 0.05)

95% confidence interval for the difference between medians [-1. -1.]


In [83]:
%%time
np.random.seed(0)

exp_mean_scores = map(np.mean, iter_bootstrap_samples(exp_slot, 1500))
control_mean_scores = map(np.mean, iter_bootstrap_samples(control_slot, 1500))

print "95% confidence interval for the exp median repair time:",  stat_intervals(exp_median_scores, 0.05)
print "95% confidence interval for the control median repair time:",  stat_intervals(control_median_scores, 0.05)

95% confidence interval for the exp median repair time: [ 5.  5.]
95% confidence interval for the control median repair time: [ 4.  4.]
CPU times: user 18.6 s, sys: 144 ms, total: 18.7 s
Wall time: 18.7 s


In [85]:
delta_median_scores = map(lambda x: x[1] - x[0], zip(exp_mean_scores, control_mean_scores))
print "95% confidence interval for the difference between medians",  stat_intervals(delta_median_scores, 0.05)

95% confidence interval for the difference between medians [-0.41172295 -0.20395125]
